# Assignment Hyperparameter Optimization

In [ ]:
# !pip uninstall tf-keras
# !pip install keras-tuner
# !pip install tensorflow==2.16.1

In [ ]:
import keras
import tensorflow as tf
print("Keras Current Version:", keras.__version__, "Tensorflow Current Version:", tf.__version__)

Keras Current Version: 3.8.0 Tensorflow Current Version: 2.18.0


# Imports

In [ ]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.1 MB/s eta 0:00:00


In [ ]:
import keras_tuner as kt

In [ ]:
import numpy as np
import pandas as pd
from joblib import dump, load
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.initializers import RandomNormal, RandomUniform, GlorotUniform, GlorotNormal, HeNormal
from keras.optimizers.schedules import ExponentialDecay
from keras_tuner import RandomSearch, GridSearch, BayesianOptimization
from keras_tuner.engine.hyperparameters import HyperParameters

random.seed(46)
np.random.seed(46)
tf.random.set_seed(46)

# import os
import time


# Functions

In [ ]:
def preprocess_data(filepath):
    data = pd.read_csv(filepath)
    scaler = StandardScaler()
    X = scaler.fit_transform(data.drop('Outcome', axis=1))
    y = data['Outcome'].values
    dump(scaler, 'scaler.joblib')
    return X, y

def prepare_datasets(X_train, X_val, y_train, y_val, batch_size=None):
    if batch_size is None:
        batch_size = len(X_train)
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(batch_size)
    val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
    val_dataset = val_dataset.batch(batch_size)
    return train_dataset, val_dataset

def plot_training_history(history, train_loss='loss', train_metric='accuracy', val_loss='val_loss', val_metric='val_accuracy'):

    #Loss
    plt.figure(figsize=(10, 5))
    plt.plot(history.history[train_loss], label='Training Loss')
    plt.plot(history.history[val_loss], label='Validation Loss')
    plt.title('Training and Validation Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    # Metrics
    plt.figure(figsize=(10, 5))
    plt.plot(history.history[train_metric], label=f"Training: {train_metric}")
    plt.plot(history.history[val_metric], label=f"Validation: {val_metric}")
    plt.title(f'Training and Validation {train_metric} Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel(f'train_metric')
    plt.legend()
    plt.show()

def get_best_epoch_details(history):
    val_losses = history.history['val_loss']
    min_val_loss_index = val_losses.index(min(val_losses))
    best_epoch = min_val_loss_index + 1

    epoch_details = {}
    for key in history.history.keys():
        epoch_details[key] = history.history[key][min_val_loss_index]

    epoch_details['best_epoch'] = best_epoch
    print(f"Best epoch details: {epoch_details}")

# Data Preparation

In [ ]:
X, y = preprocess_data('diabetes.csv')

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

train_ds, val_ds = prepare_datasets(X_train, X_val, y_train, y_val, batch_size=32)

In [ ]:
def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(train_ds.element_spec[0].shape[1],)))

    # Hidden layers, activation functions, l2, Dropout
    for i in range(hp.Int("num_layers", min_value=1, max_value=10)): # 1-10 layer

        model.add(Dense(units=hp.Int(f"units{i}", min_value=32, max_value=512, step=16), # 2-512 arasında 16'şar
                        activation=hp.Choice('activation_' + str(i), values=["relu", "sigmoid", "tanh"]), # Aktivasyon fonksiyonları relu, tanh, sigmoid
                        kernel_regularizer=l2(hp.Float(f"l2_{i}", min_value=1e-4, max_value=1e-2, sampling="log")))) #  0.0001-0.01

        model.add(BatchNormalization())
        model.add(Dropout(hp.Float(f"dropout{i}", min_value=0.1, max_value=0.5, step = 0.05))) # 0.1-0.5 arasında 0.05

    model.add(Dense(1, activation='sigmoid'))

    # Learning rate schedule
    initial_learning_rate = hp.Float("initial_learning_rate", min_value = 1e-4, max_value = 1e-2, sampling="log") # 0.0001-0.01 (1e-4 - 1e-2)

    lr_schedule = ExponentialDecay(
        initial_learning_rate=initial_learning_rate,
        decay_steps = 20, #  20
        decay_rate=0.96,
        staircase=True
    )

    # optimizers
    optimizer_choice = hp.Choice('optimizer', values=['sgd', 'adam', "rmsprop"])
    if optimizer_choice == 'sgd':
        optimizer = SGD(
            learning_rate=lr_schedule,
            momentum=hp.Float('momentum', min_value=0.0, max_value=0.9, step=0.1)
        )
    elif optimizer_choice == 'adam':
        optimizer = Adam(
            learning_rate=lr_schedule,
            beta_1=hp.Float('beta1', min_value=0.85, max_value=0.99, step=0.01),
            beta_2=hp.Float('beta2', min_value=0.999, max_value=0.9999, step=0.0001),
            epsilon=hp.Float('epsilon', min_value=1e-8, max_value=1e-7, step=1e-8)
        )

    elif optimizer_choice == 'rmsprop':
        optimizer = RMSprop(
            learning_rate=lr_schedule,
            rho=hp.Float('rho', min_value=0.8, max_value=0.99, step=0.01),
            epsilon=hp.Float('epsilon', min_value=1e-10, max_value=1e-8, step=1e-10),
            momentum=hp.Float('momentum', min_value=0.0, max_value=0.9, step=0.1)
        )

    model.compile(optimizer=optimizer,
                  loss="binary_crossentropy",
                  metrics=["accuracy"])

    return model

In [ ]:
tunner = kt.RandomSearch(
    build_model,
    objective = "val_loss",
    max_trials = 50,
    directory = "kt_random",
    project_name= "diabets",
    overwrite = True
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

tunner.search(
    train_ds,
    validation_data=val_ds,
    epochs=200,
    callbacks=[early_stopping]
)

Trial 50 Complete [00h 00m 19s]
val_loss: 0.6639091968536377

Best val_loss So Far: 0.5059324502944946
Total elapsed time: 00h 22m 20s


In [ ]:
top_3 = tunner.get_best_hyperparameters(num_trials=3)

set 1 için öğreneğin  başlangıç activation function ı sigmoid ve sonra relu tanh karışık şekilde  gitmiş en son çıktı katmanında da yine tanh kullanılmış . ilk katmanda 416 nöron varmış. en son katmanda da 400 nöron varmış.

In [ ]:

import json, os
os.makedirs("best_models", exist_ok=True)

for i, hp in enumerate(top_3, start=1):
    with open(f"best_models/hp_{i}.json", "w") as f:
        json.dump(hp.values, f, indent=2)

    print(f"\n––– Hyper-parameter set #{i} –––")
    for k, v in hp.values.items():
        print(f"{k:>18}: {v}")



––– Hyper-parameter set #1 –––
        num_layers: 1
            units0: 416
      activation_0: sigmoid
              l2_0: 0.0010339633001967396
          dropout0: 0.2
initial_learning_rate: 0.004256409192507239
         optimizer: rmsprop
          momentum: 0.2
            units1: 64
      activation_1: relu
              l2_1: 0.00021091169566063192
          dropout1: 0.4
            units2: 208
      activation_2: tanh
              l2_2: 0.00023732836845124968
          dropout2: 0.30000000000000004
            units3: 144
      activation_3: relu
              l2_3: 0.00017168785604646604
          dropout3: 0.45000000000000007
            units4: 32
      activation_4: relu
              l2_4: 0.0010748080614242867
          dropout4: 0.2
            units5: 384
      activation_5: tanh
              l2_5: 0.00011687281140446384
          dropout5: 0.35
            units6: 176
      activation_6: tanh
              l2_6: 0.0002875309185845335
          dropout6: 0.300000000

In [ ]:
best_model = tunner.get_best_models(num_models=3)

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 1 variables whereas the saved optimizer has 13 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 1 variables whereas the saved optimizer has 13 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
for i, model in enumerate(best_model, start=1):
    model.save(f"best_models/model_{i}.h5")

In [ ]:
print("\nValidation performance:")
for i, model in enumerate(best_model, start=1):
    loss, acc = model.evaluate(val_ds, verbose=0)
    print(f"Model {i}:  val_loss = {loss:.4f}   |   val_acc = {acc:.4f}")


Validation performance:
Model 1:  val_loss = 0.5059   |   val_acc = 0.7727
Model 2:  val_loss = 0.5078   |   val_acc = 0.8052
Model 3:  val_loss = 0.5223   |   val_acc = 0.7662


Val _loss a göre sıralandığı için ve en düşük kayıp da her zaman en yüksek accuracy anlamına gelmediği için.